Reference: https://deepctr-torch.readthedocs.io/en/latest/Examples.html

In [19]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, get_feature_names

# Models
import torch
from deepctr_torch.models import DeepFM

# Tuning 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#### Importing Data

In [11]:
data = pd.read_csv("../../Data Files/Raw Data/users_ratings.csv")

#### Label Encoding for sparse features

In [12]:
sparse_features = ["Title", "Year", "Directors", "Genre", "UserID", "Movie_Rating"]

target = ['User_Rating']

for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique()) for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

#### Generate input data for model

In [13]:
train, test = train_test_split(data, test_size=0.2)

x_train = {name: train[name] for name in feature_names}

x_test = {name: test[name] for name in feature_names}

y_train = train[target].values

y_test = test[target].values

 #### Defining DeepFM Architecture

In [14]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)
model.compile("adam", "mse", metrics=['mse'],)

In [ ]:
history = model.fit(x_train, y_train, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

#### Training DeepFM

In [16]:
pred_ans = model.predict(x_test, batch_size=256)

In [17]:
print("test MSE", round(mean_squared_error(y_test, pred_ans), 4))

test MSE 2.4055


#### Generating Predictions

#### Saving Model File and Predictions

In [ ]:
save_path = '../../Data Files/'
pickle.dump(model, open(save_path + 'Model Files/' + 'deepfm.pkl', 'wb'))
np.savetxt(save_path + 'Predictions/' + 'deepfm_output.csv', y_pred_dfm, delimiter=",")